# 프로젝트2 - 스마트카드 데이터 대중교통망 Circuity 활용

### 문제 제시 배경

* 최근 도시 팽창으로 인해 교통혼잡이 날로 증가하여 사회 문제로 부상하고 있다. 이를 해결하기 위해 '통합거리 비례제'를 실시해 버스, 지하철, 버스-지하철 이러한 환승통행이 이루어지고 있다. 이에 대한 해결책의 일환으로 시장에서는 고객의 요구와 수요에 대응하는 '주문형 서비스'인 '온디맨드 서비스'가 생겨나고 있다. 이 서비스에 대한 논의가 활발하게 이루어지고 있는 와중에, 이를 활용한 MaaS 플랫폼이 다수 출시되고 있다. 서울시 내 혼잡지역을 파악해 어느 지역에 '온디맨드 서비스'를 실시하면 좋을지, 궁극적으로 어떻게 하면 교통혼잡을 줄일 수 있을지 라는 생각으로 이 프로젝트를 시작하였다.

### 프로젝트 개요

* 수요와 공급에 따른 교통수단별 Circuity(우회계수)를 확인하고, 취약지점을 파악해 수요가 필요한 지점에 서비스를 하는 것이 중요하다.

### 프로젝트 목적

* Smart Card 데이터를 활용하여 서울특별시 내의 대중교통 취약지점을 확인하고, 온디맨드 서비스의 최적 입지에 대해 분석한다.

### 분석 절차

1. 실제 통행 기록인 스마트 카드 데이터를 활용하여 서울시 내 대중교통 취약지점을 파악한다. 이를 활용해 온디맨드 서비스가 어느 지점에서 실시되면 좋을지 파악한다.
* 데이터를 정제하고, 지도 상에 시각화하여 서울시 내 25개 구에 대한 Circuity 빈도를 파악한다.

2. 시각화를 통해 어느 지역을 중심으로 파악하면 좋을지 생각해본다. Circuity(우회계수)의 빈도가 높은 4개의 구를 중심으로 세부적인 내용을 분석한다.
* 버스, 지하철, 버스-지하철 그룹으로 나눠 이에 대한 분석을 진행한다.
* 위도, 경도 데이터를 활용한 대중교통 이용 고객의 루트를 확인한다.
* 경로 검색 데이터를 시계열적으로 분석하면 교통수단 및 이용 장소 등의 변화를 유추할 수 있을 것이다.

3. 각 데이터의 분석결과를 전처리 및 시각화 과정을 통해 Circuity와의 관계성을 파악한다.

### 예상 기대효과

* 혼잡한 지점, 평균 대기시간이 긴 지점 주변을 공략하면 인원이 분산되는 효과를 얻을 수 있을 것이다.
* 대중교통이 미비한 지점, 역과의 거리가 너무 먼 지점 등은 택시 등을 이용하게 되므로, 이 지점에 온디맨드 서비스가 생긴다면 효과를 볼 수 있을 것이다.
* Circuity가 큰 지점은 그만큼 우회해서 간다는 뜻이므로 시간이 오래 걸린다. Circuity와 관련된 분석을 통해 소비자의 요구를 효율적으로 해결할 수 있는 온디맨드 서비스가 필요한 지점을 파악하여 소비자의 수요와 불편을 해결할 수 있을 것이다.

### 해야할일

1. 시군구별로 데이터를 분리하기
2. Circuity가 높은 4개의 시군구를 중심으로 데이터를 분석하기.. 시군구명칭 pivot_table
 - 서초구, 양천구, 중랑구, 서대문구를 중심으로 데이터 분석.
 - 유의미한 Circuity 데이터 추출을 위해서
 - 대중교통이 취약한 부분은 혼잡하고, 돌아가는 지점 . Circuity > 1.0
 - Circuity가 1.0보다 아래인 지점은 정제
 - 거시적으로는 문제 배경을 잡는 것을 중심으로, 미시적으로는 자세하게 분석.
 - 특정 행정동의 혼잡 경로 중심으로 분석할 것이므로, total distance를 적게 잡을 필요성 있음.
 - crosstab으로 특정 구에 해당하는 데이터만 원본데이터에서 추출하고, 경로를 특정하자.
 - Circuity가 3.0이상인 데이터에 대해서는 고려하지 않음(why? 극단적으로 높은 Circuity는 분류해서 확인할 필요가 있고, 오류로 인해 높은 경우가 있음)
3. 어떤 관계성을 데이터 분석할까?
4. 대중교통 이용승객 수와 도로교통별 혼잡 우회계수와 상관관계가 있을까??

## 1. 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('font',family='Malgun Gothic')
mpl.rc('axes',unicode_minus=True)
pd.options.display.max_columns=100
pd.options.display.max_rows=100
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import datetime
import json
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

pd.options.display.max_columns=100
pd.options.display.max_rows=100

## 2. 데이터 불러오기

In [2]:
df=pd.read_csv('C:/Users/user/Desktop/TodayLearn/TiL/Code/Project2/Projcet02/1_smart_card_data/'',encoding='euc-kr')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/user/Desktop/프로젝트_2차/1안_스마트카드데이터/smartcard_Chain.csv'

In [ ]:
print(df.shape)

In [ ]:
df.isnull().sum()

## 3. 데이터 정제
* df:원본데이터
* df_copy=df.copy()

### 3.1. 결측치 및 이상치 확인, 외부 데이터 병합

In [ ]:
df_copy=df.copy()

In [ ]:
df_copy

In [ ]:
df_copy.info()

In [ ]:
df_copy['Circuity']

In [ ]:
df_copy['Circuity']=np.where(df_copy['Circuity']==0,np.nan,df_copy['Circuity'])
df_copy['Circuity']=np.where(df_copy['Haversine']==0,np.nan,df_copy['Circuity'])

In [ ]:
df_copy.isnull().sum()

In [ ]:
df_copy.shape

In [ ]:
df_copy['route_id']=df_copy['route_id'].fillna('-1')
df_copy.isnull().sum()

어떤 것을 분석하면 좋을까?
Circuity를 활용할 수 있는 방법은? 탑승 시간에 따른 평균 Circuity의 변화를 확인하자.
total distance/Haversine=Circuity
Circuity가 1에 가까울수록 직전성이 높다, 1보다 클수록 돌아간다는 의미
차량 공유 서비스는 근처에 있어야한다.
-짧은 거리에도 버스, 지하철이 아닌 빠르게 갈 수 있는 공유 자전거, 공유 킥보드
-교통수단별 수요 파악
-이용자의 카결제 데이터를 파악하여 교통수단별 지역간 수요(출발지-도착지 분포,OD 분포)OD분포-origin destination,중심 시가지와 주변지역의 수요가 많다(노선버스),택시.대여 자전거는 중심 시가지내에 단거리 이동에 많이 이용.
-경로검색 데이터를 시계열적으로 분석 시, 교통수단 및 이동장소 등의 변화를 유추할 수 있음.
-ICT,AI 등의 최신 기술을 도입하여 누구나 이용할 수 있는 새로운 이동서비스를 제공

In [ ]:
#시간대 변화
#40키로 되는 거리는 차량을 이용하지 굳이 온디맨드 서비스를 이용할까?,,온디맨드 서비스는 가까운 거리에서 이용하지 않을까,,,

#시간대-분단위로 변화(total_time이 초 단위로 되어있으니까)
df_copy['total_time(m)']=(df_copy['total_time']//60)
df_copy.head()

평균 이동거리에 걸린 시간대비 Circuity가 차이가 있는지에 대해서 확인
차량공유 서비스가 아니라 전동킥보다, 자전거 같은 종류는 상대적으로 먼 거리에서 사용 안함.
그럼 total distance를 2키로 정도로 한정할 수 있지 않을까?

Circuity가 적은 곳에선 안 쓰고
Circuity가 많은 곳에선 쓰고(우회계수가 높을수록 우회해서 간다. 더 돌아간다)

버스는 노선이 한정되어 있어서 가려면 돌아가야 됨. 사람들이 어떤 곳에서 차량공유를 할까? 상대적으로 긴 거리에선 오랜 시간이 걸리고, 우회계수가 높은 곳을 갈 때 차량공유를 할 것이다. 아니면 버스가 안 다니는 곳을 갈 때 차량공유를 할 것이다.

In [ ]:
#시군구명칭.csv 파일 병합(외부자료)
#on_area_name 탑승위치가 해당되는 구의 명칭을 데이터에 통합
sample=pd.read_csv('C:/Users/user/Desktop/프로젝트_2차/1안_스마트카드데이터/seoul_id.csv',encoding='euc-kr',dtype='object')

sample.rename(columns={'행정구역번호':'on_dong'},inplace=True)
sample.head()

In [ ]:
df_copy['on_dong']=df_copy['on_dong'].astype(str)

In [ ]:
df_copy=df_copy.merge(sample,how='left',on='on_dong')

In [ ]:
df_copy.isnull().sum()

In [ ]:
df_copy.dropna(inplace=True)

In [ ]:
df_copy.isnull().sum()

In [ ]:
df_copy.tail(3)

In [ ]:
df_copy['시군구명칭'].value_counts()

### 3.2. 특정 구간 데이터 확인을 위한 Circuity 구간 보정

* total_distance가 너무 큰 데이터는 총 거리가 먼데, 온디맨드 서비스를 이용할까에 대한 의문이 들어 제외하고, Circuity(우회계수)가 1보다 작은 것은 직선거리가 더 큰 것이므로 우리가 하고자 하는 프로젝트의 insight와 안 맞다고 판단해 제외한다. 그리고 Circuity가 3보다 큰 데이터도 그래프를 그려보면 유의미한 값과는 거리가 멀다고 판단해 이 또한 제외한다.

In [ ]:
df_copy=df_copy[(df_copy['Circuity']>1.0)&(df_copy['Circuity']<3.0)]
print(df_copy.shape)

In [ ]:
df_copy.head()

In [ ]:
#plt.figure(figsize=(20,10))
plt.xticks(rotation=45)
sns.barplot(data=df_copy,x='시군구명칭',y='Circuity')
plt.show()

## 4. 데이터 분석 과정

### 4.1. Folium, 우회계수를 활용한 시군구에 따른 혼잡도 파악

In [ ]:
import folium

folium.Map()

- 시군구별로 데이터 분리하기
- Circuity가 높은 4개의 시군구를 중심으로 데이터 분석
- 관계성 찾아내기

In [ ]:
import json

In [ ]:
geo_path='C:/Users/user/Desktop/프로젝트_2차/1안_스마트카드데이터/municipalities-geo-simple (1).json'
geo_str=json.load(open(geo_path,encoding='utf-8'))

center=[37.541,126.986]
m1=folium.Map(location=center,zoom_start=10.5)

folium.Choropleth(
        geo_data=geo_str,
        data=df_copy,
        columns=('시군구명칭','Circuity'),
        key_on='feature.properties.name',
        fill_color='BuPu',
        legend_name='탑승빈도').add_to(m1)

m1

* Folium 데이터 단계구분도 확인 결과, 상위 30% 내에서 서초구, 중랑구, 양천구, 서대문구 총 4개의 구역에서 우회계수의 분포가 높게 나타남.- 타 구에 비해 혼잡하다고 추정.
* 이를 기준으로 4군데의 지역의 데이터를 분석해 유의미한 insight를 찾아내는 것이 프로젝트의 목적이다.

* 서초구 
* 중랑구
* 양천구 
* 서대문구 

In [ ]:
df_copy.pivot_table(index='시군구명칭',values='Circuity',aggfunc='mean').T

* 서초구 Circuity 평균이 1.284
* 중랑구 Circuity 평균이 1.271
* 양천구 Circuity 평균이 1.330
* 서대문구 Circuity 평균이 1.363

* 4개의 구를 각각 비교하여 혼잡정도를 비교, 최종 온디맨드 서비스 최적위치를 확인함.

In [ ]:
plt.figure(figsize=(20,14))
plt.xticks(rotation=45)
sns.boxplot(data=df_copy,x='시군구명칭',y='Circuity')

In [ ]:
df_copy_seocho=df_copy[df_copy['시군구명칭']=='서초구']
df_copy_seocho

In [ ]:
df_copy_joong=df_copy[df_copy['시군구명칭']=='중랑구']
df_copy_joong

In [ ]:
df_copy_yang=df_copy[df_copy['시군구명칭']=='양천구']
df_copy_yang

In [ ]:
df_copy_seodae=df_copy[df_copy['시군구명칭']=='서대문구']
df_copy_seodae

### 4.2. 통행수단에 따른 Circuity의 추이

### 그룹마다 Circuity 비교
- 버스랑 지하철 둘 다 타는 그룹
- 버스만 타는 그룹
- 지하철만 타는 그룹

In [ ]:
df_copy.head()

In [ ]:
cond1=(df_copy['Bus']==True)&(df_copy['Subway']==False)

df_bus = df_copy[cond1]

In [ ]:
df_bus

In [ ]:
cond2=(df_copy['Bus']==False)&(df_copy['Subway']==True)

df_subway=df_copy[cond2]

In [ ]:
df_subway

In [ ]:
cond3=(df_copy['Bus']==True)&(df_copy['Subway']==True)

df_both=df_copy[cond3]

In [ ]:
df_both

In [ ]:
group_tranfer={'only_bus':df_bus['Circuity'].mean(),
    'only_subway':df_subway['Circuity'].mean(),
    'both':df_both['Circuity'].mean()
}

group_tranfer

* df_copy에서 그룹을 나눠 버스만 이용한 경우 Circuity 평균은 1.335이고, 지하철만 이용한 경우 Circuity 평균은 1.230이고, 버스와 지하철 둘 다 이용한 경우 Circuity 평균은 1.348이다. 이것을 기준으로 데이터를 절삭할 수 있다.

* 몇 개의 데이터를 살펴보니, 일이 있어 일부러 돌아간다거나 잘못 타서 내리고 다시 타는 경우 등 이러한 경우가 있어 이러한 데이터도 제외시켜야 할 것 같다.
* haversine은 직선거리인데, haversine이 total_distance보다 큰 경우도 말이 안 된다.

In [ ]:
df_mean=pd.DataFrame(columns=['name','mean'],data=group_tranfer.items())
df_mean

In [ ]:
df_mean=pd.pivot_table(data=df_mean,index='name',values='mean')
df_mean

In [ ]:
df_mean.plot(kind='bar')

### 4.3. 특정 4개의 구에 대한 개별적 분석 실시

### 4.3.0. 서초구, 중랑구, 양천구, 서대문구의 평균 Circuity 확인

In [ ]:
df_sel=df_copy[df_copy['시군구명칭'].str.contains('서초구|중랑구|양천구|서대문구')]
df_sel.head()

In [ ]:
print(df_sel.shape)

* 백만개의 데이터 분석

In [ ]:
sns.barplot(data=df_sel,x='시군구명칭',y='Circuity')

In [ ]:
#sns.barplot(data=df_sel,x='total_distance',y='Circuity',hue='시군구명칭')

### 4.3.1. 서초구의 대중교통 취약지점 분석

In [ ]:
df_copy_seocho.head()

In [ ]:
df_copy_seocho.describe()

* 서초구의 total_distance 평균이 8627.74m (total_distance<=2000)-너무 큰 데이터는 제외
* 서초구에서 버스만 타는 그룹, 지하철만 타는 그룹, 버스랑 지하철 둘 다 타는 그룹
* haversine이 total_distance보다 큰 경우도 봐야함.

### 4.3.2. 양천구의 대중교통 취약지점 분석

### 4.3.3. 중랑구의 대중교통 취약지점 분석

In [ ]:
print(df_copy_joong.shape)

In [ ]:
df_copy_joong.head(2)

* 중랑구에서 버스만 타는 그룹, 지하철만 타는 그룹, 버스-지하철 둘 다 타는 그룹

In [ ]:
only_bus=(df_copy_joong['Bus']==True)&(df_copy_joong['Subway']==False)
only_subway=(df_copy_joong['Bus']==False)&(df_copy_joong['Subway']==True)
both=(df_copy_joong['Bus']==True)&(df_copy_joong['Subway']==True)

In [ ]:
df_joong_bus=df_copy_joong[only_bus]
df_joong_sub=df_copy_joong[only_subway]
df_joong_both=df_copy_joong[both]

In [ ]:
df_joong_sub[df_joong_sub['Circuity']>1.5]

In [ ]:
Is_dong=list(set(list(df_copy_joong['읍면동명칭'])))

Is_dong

In [ ]:
df_copy_joong.head()

In [ ]:
df_joong=df_copy_joong.copy()
df_joong.head()

In [ ]:
df_joong['on_lat'].mean()

In [ ]:
df_joong['on_lon'].mean()

In [ ]:
import random
i=random.randint(0,1000)

print(i)

#### 4.3.3.1. 면목 3,8동 내 이용승객 분포

In [ ]:
Is_dong=list(set(list(df_joong['읍면동명칭'])))

Is_dong

In [ ]:
n=len(Is_dong)

for i in range(0,len(Is_dong)):
    df_joong[df_joong['읍면동명칭']==Is_dong[i]]

In [ ]:
df_joong_meon3=df_joong[df_joong['읍면동명칭']==Is_dong[12]]

df_joong_meon3.head()

In [ ]:
def find_circuity(data):
    cond=(df_joong['Circuity']>1.5)&(df_joong['total_distance']<=2000)
    return data[cond]

* 우회계수가 높은 지역에 온디맨드 서비스가 필요할 것이고, 거리는 이 프로젝트에서 하고자 하는 온디맨드 서비스 즉, 전동 퀵보드와 같은 짧은 거리에서 유용하게 사용할 것 같은 온디맨드 서비스를 제공하기 위해 2km로 제한함.

In [ ]:
df_joong_meon3_cir=find_circuity(df_joong_meon3)

print(df_joong_meon3_cir.shape)
df_joong_meon3_cir.head()

* 대중교통 이용한 사람 기준

In [ ]:
df_joong_meon3['on_lat'].mean()

In [ ]:
df_joong_meon3['on_lon'].mean()

In [ ]:
df_joong_meon3_cir['on_lat'].mean()

In [ ]:
import folium
import random

map=folium.Map(location=[37.58297,127.09096],zoom_start=12)

from folium.plugins import MarkerCluster
marker_cluster=MarkerCluster().add_to(map)

for lat, lon in zip(df_joong_meon3['on_lat'],df_joong_meon3['on_lon']):
    folium.Marker([lat,lon]).add_to(marker_cluster)
    
map

* 범위 조정한 데이터로 대중교통 이용한 사람, 도착한 사람

In [ ]:
import folium
import random

map=folium.Map(location=[37.58297,127.09096],zoom_start=12)

from folium.plugins import MarkerCluster
marker_cluster=MarkerCluster().add_to(map)

for lat, lon in zip(df_joong_meon3_cir['on_lat'],df_joong_meon3_cir['on_lon']):
    folium.Marker([lat,lon]).add_to(marker_cluster)
    
for lat, lon in zip(df_joong_meon3_cir['off_lat'],df_joong_meon3_cir['off_lon']):
    folium.Marker([lat,lon]).add_to(marker_cluster)
    
map

* 상봉터미널(134건), 하나인쇄명함할인마트(216건), 서일대학교(500건 이상)

* 범위 조정한 데이터로 대중교통 도착한 사람

In [ ]:
import folium
import random

map=folium.Map(location=[37.58297,127.09096],zoom_start=12)

from folium.plugins import MarkerCluster
marker_cluster=MarkerCluster().add_to(map)

for lat, lon in zip(df_joong_meon3_cir['off_lat'],df_joong_meon3_cir['off_lon']):
    folium.Marker([lat,lon]).add_to(marker_cluster)
    
map

#### 4.3.3.2. 면목2동 내 이용승객 분포

In [ ]:
df_joong_meon2=df_joong[df_joong['읍면동명칭']==Is_dong[3]]
print(df_joong_meon2.shape)
df_joong_meon2.head()

In [ ]:
df_joong_meon2_cir=find_circuity(df_joong_meon2)

print(df_joong_meon2_cir.shape)
df_joong_meon2_cir.head()

In [ ]:
clustering_whole(df_joong_meon2)

In [ ]:
clustering_sub(df_joong_meon2_cir)

* 면목공원 근처 7호선 면목역(151건), 중랑초등학교-면목한신아파트 근처(113건)

#### 4.3.3.3. 면목4동 내 이용승객 분포

In [ ]:
df_joong_meon4=df_joong[df_joong['읍면동명칭']==Is_dong[8]]
print(df_joong_meon4.shape)
df_joong_meon4.head()

In [ ]:
df_joong_meon4_cir=find_circuity(df_joong_meon4)

print(df_joong_meon4_cir.shape)
df_joong_meon4_cir.head()

In [ ]:
clustering_whole(df_joong_meon4)

In [ ]:
clustering_sub(df_joong_meon4_cir)

#### 4.3.3.4. 면목5동 내 이용승객 분포

In [ ]:
df_joong_meon5=df_joong[df_joong['읍면동명칭']==Is_dong[15]]
print(df_joong_meon5.shape)
df_joong_meon5.head()

In [ ]:
df_joong_meon5_cir=find_circuity(df_joong_meon5)

print(df_joong_meon5_cir.shape)
df_joong_meon5_cir.head()

In [ ]:
clustering_whole(df_joong_meon5)

In [ ]:
clustering_sub(df_joong_meon5_cir)

#### 4.3.3.5. 면목7동 내 이용승객 분포

In [ ]:
df_joong_meon7=df_joong[df_joong['읍면동명칭']==Is_dong[6]]
print(df_joong_meon7.shape)
df_joong_meon7.head()

In [ ]:
df_joong_meon7_cir=find_circuity(df_joong_meon7)

print(df_joong_meon7_cir.shape)
df_joong_meon7_cir.head()

In [ ]:
clustering_whole(df_joong_meon7)

In [ ]:
clustering_sub(df_joong_meon7_cir)

* 버스 노선 따라 돌아감. 장안교 건너서 가는 버스 노선

#### 4.3.3.6. 면목본동 내 이용승객 분포

In [ ]:
df_joong_meonbon=df_joong[df_joong['읍면동명칭']==Is_dong[11]]
print(df_joong_meonbon.shape)
df_joong_meonbon.head()

In [ ]:
df_joong_meonbon_cir=find_circuity(df_joong_meonbon)

print(df_joong_meonbon_cir.shape)
df_joong_meonbon_cir.head()

In [ ]:
clustering_whole(df_joong_meonbon)

In [ ]:
clustering_sub(df_joong_meonbon_cir)

* 서일대학교 주변(300건 이상), 서울중목초등학교 앞/면목한신아파트 앞(100건 이상), 지하철 7호선 면목역-하나인쇄명함할인마트(719건)

#### 4.3.3.7. 묵1동 내 이용승객 분포

In [ ]:
df_joong_mook1=df_joong[df_joong['읍면동명칭']==Is_dong[5]]
print(df_joong_mook1.shape)
df_joong_mook1.head()

In [ ]:
df_joong_mook1_cir=find_circuity(df_joong_mook1)

print(df_joong_mook1_cir.shape)
df_joong_mook1_cir.head()

In [ ]:
clustering_whole(df_joong_mook1)

In [ ]:
clustering_sub(df_joong_mook1_cir)

* 봉화산을 둘러서 가야하니까 우회계수 높음.

#### 4.3.3.8. 묵2동 내 이용승객 분포

In [ ]:
df_joong_mook2=df_joong[df_joong['읍면동명칭']==Is_dong[2]]
print(df_joong_mook2.shape)
df_joong_mook2.head()

In [ ]:
df_joong_mook2_cir=find_circuity(df_joong_mook2)

print(df_joong_mook2_cir.shape)
df_joong_mook2_cir.head()

In [ ]:
clustering_whole(df_joong_mook2)

In [ ]:
clustering_sub(df_joong_mook2_cir)

* 먹골역 주변, 화랑로 건너서 석계역 주변(100건 이상)

#### 4.3.3.9. 상봉1동 내 이용승객 분포

In [ ]:
df_joong_sang1=df_joong[df_joong['읍면동명칭']==Is_dong[1]]
print(df_joong_sang1.shape)
df_joong_sang1.head()

In [ ]:
df_joong_sang1_cir=find_circuity(df_joong_sang1)

print(df_joong_sang1_cir.shape)
df_joong_sang1_cir.head()

In [ ]:
clustering_whole(df_joong_sang1)

In [ ]:
clustering_sub(df_joong_sang1_cir)

* 상봉지하차도, 신내지하차도

#### 4.3.3.10. 상봉2동 내 이용승객 분포

In [ ]:
df_joong_sang2=df_joong[df_joong['읍면동명칭']==Is_dong[14]]
print(df_joong_sang2.shape)
df_joong_sang2.head()

In [ ]:
df_joong_sang2_cir=find_circuity(df_joong_sang2)

print(df_joong_sang2_cir.shape)
df_joong_sang2_cir.head()

In [ ]:
clustering_whole(df_joong_sang2)

In [ ]:
clustering_sub(df_joong_sang2_cir)

* 상봉터미널, 서일대학교

#### 4.3.3.11. 망우본동 내 이용승객 분포

In [ ]:
df_joong_mangbon=df_joong[df_joong['읍면동명칭']==Is_dong[0]]
print(df_joong_mangbon.shape)
df_joong_mangbon.head()

In [ ]:
df_joong_mangbon_cir=find_circuity(df_joong_mangbon)

print(df_joong_mangbon_cir.shape)
df_joong_mangbon_cir.head()

In [ ]:
clustering_whole(df_joong_mangbon)

In [ ]:
clustering_sub(df_joong_mangbon_cir)

* 중랑구청, 이화여대병설미디어고등학교, 송곡고등학교, 송곡여자중학교 근처에

#### 4.3.3.12. 망우3동 내 이용승객 분포

In [ ]:
df_joong_mang3=df_joong[df_joong['읍면동명칭']==Is_dong[7]]
print(df_joong_mang3.shape)
df_joong_mang3.head()

In [ ]:
df_joong_mang3_cir=find_circuity(df_joong_mang3)

print(df_joong_mang3_cir.shape)
df_joong_mang3_cir.head()

In [ ]:
clustering_whole(df_joong_mang3)

In [ ]:
clustering_sub(df_joong_mang3_cir)

* 면일초등학교, 혜원여자중고등학교 앞

#### 4.3.3.13. 신내1동 내 이용승객 분포

In [ ]:
df_joong_sin1=df_joong[df_joong['읍면동명칭']==Is_dong[9]]
print(df_joong_sin1.shape)
df_joong_sin1.head()

In [ ]:
df_joong_sin1_cir=find_circuity(df_joong_sin1)

print(df_joong_sin1_cir.shape)
df_joong_sin1_cir.head()

In [ ]:
clustering_whole(df_joong_sin1)

In [ ]:
clustering_sub(df_joong_sin1_cir)

* 봉화산, 신내차량사업소, 용마산로 거쳐서 가야함. 동성 1,2차 아파트, 데시앙 아파트 돌아서 가야함.

#### 4.3.3.14. 신내2동 내 이용승객 분포

In [ ]:
df_joong_sin2=df_joong[df_joong['읍면동명칭']==Is_dong[4]]
print(df_joong_sin2.shape)
df_joong_sin2.head()

In [ ]:
df_joong_sin2_cir=find_circuity(df_joong_sin2)

print(df_joong_sin2_cir.shape)
df_joong_sin2_cir.head()

In [ ]:
clustering_whole(df_joong_sin2)

In [ ]:
clustering_sub(df_joong_sin2_cir)

* 봉화산, 신내 단지 아파트 돌아서 가야함.

#### 4.3.3.15. 중화1동 내 이용승객 분포

In [ ]:
df_joong_joonghwa1=df_joong[df_joong['읍면동명칭']==Is_dong[13]]
print(df_joong_joonghwa1.shape)
df_joong_joonghwa1.head()

In [ ]:
df_joong_joonghwa1_cir=find_circuity(df_joong_joonghwa1)

print(df_joong_joonghwa1_cir.shape)
df_joong_joonghwa1_cir.head()

In [ ]:
clustering_whole(df_joong_joonghwa1)

In [ ]:
clustering_sub(df_joong_joonghwa1_cir)

* 코스트코, 홈플러스 앞, 장안중학교, 중흥초등학교, 7호선중화역 앞

#### 4.3.3.16. 중화2동 내 이용승객 분포

In [ ]:
df_joong_joonghwa2=df_joong[df_joong['읍면동명칭']==Is_dong[10]]
print(df_joong_joonghwa2.shape)
df_joong_joonghwa2.head()

In [ ]:
df_joong_joonghwa2_cir=find_circuity(df_joong_joonghwa2)

print(df_joong_joonghwa2_cir.shape)
df_joong_joonghwa2_cir.head()

In [ ]:
clustering_whole(df_joong_joonghwa2)

In [ ]:
clustering_sub(df_joong_joonghwa2_cir)

* 동일로 지하차도?

### 4.3.4. 서대문구의 대중교통 취약지점 분석

## 사용함수

### Circuity>1.5이고, total_distance가 2km인 데이터를 정제-find_circuity(data)

In [ ]:
def find_circuity(data):
    cond=(data['Circuity']>1.5)&(data['total_distance']<=2000)
    return data[cond]

### 이용승객 클러스터링

In [ ]:
def clustering_whole(data):
    import folium
    
    map=folium.Map(location=[37.59576,127.08818],zoom_start=12)
    
    from folium.plugins import MarkerCluster
    marker_cluster=MarkerCluster().add_to(map)
    
    for lat, lon in zip(data['on_lat'],data['on_lon']):
        folium.Marker([lat,lon]).add_to(marker_cluster)
        
    return map

In [ ]:
def clustering_sub(data):
    import folium
    
    map=folium.Map(location=[37.59576,127.08818],zoom_start=12)
    
    from folium.plugins import MarkerCluster
    marker_cluster=MarkerCluster().add_to(map)
    
    for lat, lon in zip(data['on_lat'],data['on_lon']):
        folium.Marker([lat,lon]).add_to(marker_cluster)
        
    for lat, lon in zip(data['off_lat'],data['off_lon']):
        folium.Marker([lat,lon]).add_to(marker_cluster)
        
    return map